# Process the raw data from pyapm and bpnsdata
This notebook is the code to process the output given after processing the data with pypam and bpnsdata
For more information about this process please contact clea.parcerisas@vliz.be or check the documentation of both packages
https://lifewatch-pypam.readthedocs.io/en/latest/
https://github.com/lifewatch/bpnsdata

In [1]:
import pathlib

import geopandas
import numpy as np
import pandas as pd
import xarray
from tqdm import tqdm

C:\Users\cleap\Envs\wadden_soundscape\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [7]:
# Don't forget to add the ones you added before
ENV_LABELS = [
    'route_density',
    'season',
    'moon_phase',
    'day_moment',
    'sediment_type',
    'bathymetry',
    'tide'
]

CHUNK_LENGTH = 10.0 # This has to be the SAME one you chose in 0_create_dataset (binsize)

# Set the min and max frequencies to use (has to be a range smaller or equal to the one you selected in 0_create_dataset)
MAX_FREQ = 20000
MIN_FREQ = 100

In [8]:
# Write down which of the variables are CATEGORIES and not NUMERICAL
CATEGORICAL_VARS = ['day_moment', 'sediment_type', 'tide','seabed_habitat', 'deployment_name']
CYCLIC_VARS = ['season', 'moon_phase']

vars_dtypes = {
    'route_density': int,
    'season': int,
    'moon_phase': np.float16,
    'day_moment': 'category',
    'sediment_type': 'category',
    'bathymetry': np.float16,
    'tide': 'category',
    'deployment_name': 'category'
}


In [9]:
# Define the folders
data_path = pathlib.Path('./data/raw_data/')
processed_data_path = pathlib.Path('./data/processed/')
raw_data_path = pathlib.Path('./data/raw_data/deployments/deployments')

In [10]:
# Read the metadata csv
metadata = pd.read_csv(data_path.joinpath('data_summary_mda.csv'))

# Create the empty output vars
df_features = pd.DataFrame()
df_sample = pd.DataFrame()
df_env = pd.DataFrame()
df_geo = geopandas.GeoDataFrame()
df_labels = pd.DataFrame()

# Define the names of the vars that will be used
# HERE IMPORTANT TO DECIDE IF YOU PROCESS 'oct3' or 'millidecade_bands'
features_var = 'oct3'
deployment_columns = ['deployment_name', 'datetime']
freqticks = None

In [11]:
# Join all the deployments in one DataFrame
df = pd.DataFrame()
total_acoustic_time = 0
for idx in tqdm(metadata.index, total=len(metadata)):
    deployment_row = metadata.loc[idx]
    env_name = '%s_%s_env.nc' % (idx, deployment_row.deployment_name)
    env_path = processed_data_path.joinpath(env_name)
    deployment_file_name = '%s_%s.nc' % (idx, deployment_row.deployment_name)
    name = deployment_row['deployment_name']
    deployment = xarray.open_dataset(env_path)

    # Get the geometry
    geo_series = geopandas.GeoSeries(data=geopandas.points_from_xy(x=deployment['longitude'],
                                                                   y=deployment['latitude']),
                                     crs='EPSG:4326')

    # Eliminate the frequencies below MIN_FREQ and above MAX_FREQ
    deployment = deployment.sel(frequency=deployment.frequency[deployment.frequency < MAX_FREQ])
    deployment = deployment.sel(frequency=deployment.frequency[deployment.frequency > MIN_FREQ])
    deployment_duration = deployment.datetime.max() - deployment.datetime.min()
    total_acoustic_time += deployment_duration
    deployment = deployment[ENV_LABELS + [features_var] + deployment_columns].dropna('id', 'any')
    clean_freqticks = deployment.frequency.values

    # Create a pandas df with all the wanted values
    values_arr = deployment[features_var].values
    df_deployment = pd.DataFrame(values_arr)
    df_deployment = df_deployment.astype(np.float16)

    for env in ENV_LABELS + deployment_columns:
        df_deployment[env] = deployment[env]

    df_deployment = geopandas.GeoDataFrame(df_deployment, geometry=geo_series)

    df = pd.concat([df, df_deployment], ignore_index=True)
    deployment.close()

# print the total acoustic time
print('Total amount of time recorded %s h' % (total_acoustic_time.values / np.timedelta64(1, 'h')))

100%|██████████| 15/15 [00:00<00:00, 24.05it/s]

Total amount of time recorded 1041.0233333333333 h


## Some data clean up

In [12]:
# Change the data types to save some computational power and memory
# Some operations
df = df.replace(['Civil twilight', 'Astronomical twilight', 'Nautical twilight'], ['Twilight', 'Twilight', 'Twilight'])
df['bathymetry'] = -1 * df['bathymetry']

# Categorical vars to category for efficient storage and processing
for env, env_type in vars_dtypes.items():
    df[env] = df[env].astype(env_type)

## Save the outputs to work on with the next script

In [13]:
# Filter the deployments to skip if there were any
np.save(processed_data_path.joinpath('used_freqticks.npy'), clean_freqticks)
df.to_pickle(processed_data_path.joinpath('df_complete.pkl'))

In [14]:
df

,0,1,2,3,4,5,6,7,8,9,...,moon_phase,day_moment,sediment_type,bathymetry,tide,deployment_name,datetime,geometry,20,21
0,80.18750,81.25000,79.62500,73.62500,74.75000,74.25000,73.37500,72.81250,71.50000,71.25000,...,6.175781,Twilight,reef,2.482422,low,NLOyster1,2022-05-29 22:00:09,POINT (4.88076 53.06843),NaN,NaN
1,82.62500,82.25000,80.31250,74.50000,76.37500,75.18750,72.00000,70.75000,70.56250,70.06250,...,6.175781,Twilight,reef,2.482422,low,NLOyster1,2022-05-29 22:00:19,POINT (4.88076 53.06843),NaN,NaN
2,80.50000,80.75000,79.06250,73.25000,74.62500,73.93750,72.25000,70.75000,70.31250,69.50000,...,6.175781,Twilight,reef,2.482422,low,NLOyster1,2022-05-29 22:00:29,POINT (4.88076 53.06843),NaN,NaN
3,81.87500,81.31250,77.62500,73.18750,76.25000,74.37500,70.75000,70.00000,69.75000,70.25000,...,6.175781,Twilight,reef,2.482422,low,NLOyster1,2022-05-29 22:00:39,POINT (4.88076 53.06843),NaN,NaN
4,80.93750,81.25000,79.87500,73.87500,75.06250,73.81250,71.18750,70.31250,70.37500,70.00000,...,6.175781,Twilight,reef,2.482422,low,NLOyster1,2022-05-29 22:00:49,POINT (4.88076 53.06843),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7111,60.09375,57.78125,57.46875,56.40625,53.18750,54.53125,56.87500,58.68750,60.93750,62.71875,...,3.501953,Twilight,coarsesand,1.554688,low,CoarseSandLow,2022-06-15 22:40:19,POINT (5.20231 53.06843),66.81250,67.1875
7112,58.68750,57.03125,57.12500,55.65625,51.78125,53.37500,56.75000,59.37500,61.53125,62.40625,...,3.501953,Twilight,coarsesand,1.554688,low,CoarseSandLow,2022-06-15 22:40:29,POINT (5.20231 53.06843),67.37500,68.0625
7113,60.75000,58.28125,58.96875,55.50000,52.65625,53.71875,55.68750,58.75000,61.50000,62.87500,...,3.501953,Twilight,coarsesand,1.554688,low,CoarseSandLow,2022-06-15 22:40:39,POINT (5.20231 53.06843),66.93750,67.3125
7114,62.12500,58.25000,57.21875,54.71875,53.81250,55.37500,59.87500,68.56250,71.56250,68.93750,...,3.501953,Twilight,coarsesand,1.554688,low,CoarseSandLow,2022-06-15 22:40:49,POINT (5.20231 53.06843),68.25000,69.3750
